In [8]:
# %% [markdown]
# # Trading Strategy Analysis - Interactive Dashboard

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
from scipy.stats import skew, kurtosis, norm, jarque_bera, ttest_1samp
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
import ipywidgets as widgets
warnings.filterwarnings('ignore')

# Styling
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load and preprocess data
print("📊 Loading and preprocessing data...")
df = pd.read_csv('positions.csv')
print(f"✅ Loaded {len(df)} transactions")

# Convert dates and calculate basic columns
df['entry_time'] = pd.to_datetime(df['entry_time'])
df['exit_time'] = pd.to_datetime(df['exit_time'])
df['entry_hour'] = df['entry_time'].dt.hour
df['entry_day'] = df['entry_time'].dt.day_name()
df['entry_month'] = df['entry_time'].dt.month
df['entry_weekday'] = df['entry_time'].dt.dayofweek

# Trade duration calculations
df['trade_duration'] = df['exit_time'] - df['entry_time']
df['trade_duration_seconds'] = df['trade_duration'].dt.total_seconds()
df['trade_duration_minutes'] = df['trade_duration_seconds'] / 60
df['trade_duration_hours'] = df['trade_duration_seconds'] / 3600

# Core performance metrics
initial_balance = df['open_equity'].iloc[0]
final_balance = df['close_equity'].iloc[-1]
net_profit = final_balance - initial_balance
total_return = (net_profit / initial_balance) * 100

# Profit/Loss analysis
gross_profit = df[df['profit'] > 0]['profit'].sum()
gross_loss = abs(df[df['profit'] < 0]['profit'].sum())
profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')

winning_trades = len(df[df['profit'] > 0])
losing_trades = len(df[df['profit'] < 0])
winning_percentage = (winning_trades / len(df)) * 100

avg_win = df[df['profit'] > 0]['profit'].mean()
avg_loss = df[df['profit'] < 0]['profit'].mean()
avg_trade = df['profit'].mean()

# Advanced metrics
largest_win = df['profit'].max()
largest_loss = df['profit'].min()
win_loss_ratio = abs(avg_win / avg_loss) if avg_loss != 0 else float('inf')
expectancy = (winning_percentage/100 * avg_win) + ((1-winning_percentage/100) * avg_loss)

# Prepare time series data
df_sorted = df.sort_values('exit_time')
df_sorted['cumulative_profit'] = df_sorted['profit'].cumsum()
df_sorted['equity_curve'] = initial_balance + df_sorted['cumulative_profit']
df_sorted['rolling_max'] = df_sorted['equity_curve'].cummax()
df_sorted['drawdown'] = (df_sorted['equity_curve'] - df_sorted['rolling_max']) / df_sorted['rolling_max'] * 100
max_drawdown = df_sorted['drawdown'].min()

# Returns for risk calculations
returns = df['profit'] / initial_balance
sharpe_ratio = returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0
negative_returns = returns[returns < 0]
sortino_ratio = returns.mean() / negative_returns.std() * np.sqrt(252) if len(negative_returns) > 1 else 0
calmar_ratio = (total_return/100) / abs(max_drawdown/100) if max_drawdown != 0 else float('inf')

# Advanced risk metrics
var_95 = np.percentile(returns, 5)
var_99 = np.percentile(returns, 1)
cvar_95 = returns[returns <= var_95].mean()
ulcer_index = np.sqrt(np.mean(df_sorted['drawdown'] ** 2))

# Statistical metrics
returns_skewness = skew(returns)
returns_kurtosis = kurtosis(returns)
jb_stat, jb_pvalue = jarque_bera(returns)

# Kelly Criterion
win_prob = winning_percentage / 100
loss_prob = 1 - win_prob
kelly_fraction = (win_prob * avg_win - loss_prob * abs(avg_loss)) / (avg_win * abs(avg_loss)) if avg_win != 0 and avg_loss != 0 else 0

print(f"""
🎯 KEY PERFORMANCE METRICS:
• Total Return: {total_return:.2f}%
• Net Profit: ${net_profit:,.2f}
• Profit Factor: {profit_factor:.2f}
• Win Rate: {winning_percentage:.1f}%
• Max Drawdown: {max_drawdown:.2f}%
• Sharpe Ratio: {sharpe_ratio:.2f}

📊 ADVANCED METRICS:
• Expectancy: ${expectancy:.2f}
• Win/Loss Ratio: {win_loss_ratio:.2f}
• Kelly Fraction: {kelly_fraction:.2%}
• Sortino Ratio: {sortino_ratio:.2f}
• 95% VaR: {var_95:.3%}
""")

# %% [markdown]
# ## 1. INTERACTIVE VISUALIZATIONS

# %%
def create_interactive_charts(show_equity=True, show_drawdown=True, show_profit_dist=True, 
                            show_duration_dist=True, window_size=30):
    """Create interactive charts with customizable parameters"""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    axes = axes.flatten()
    
    # Equity Curve
    if show_equity:
        axes[0].plot(df_sorted['exit_time'], df_sorted['equity_curve'], linewidth=2, color='blue', alpha=0.8)
        axes[0].set_title('Equity Curve', fontweight='bold', fontsize=12)
        axes[0].set_xlabel('Date')
        axes[0].set_ylabel('Equity ($)')
        axes[0].grid(True, alpha=0.3)
        axes[0].tick_params(axis='x', rotation=45)
    else:
        axes[0].axis('off')
    
    # Drawdown
    if show_drawdown:
        axes[1].fill_between(df_sorted['exit_time'], df_sorted['drawdown'], 0, 
                           where=df_sorted['drawdown'] < 0, color='red', alpha=0.3)
        axes[1].plot(df_sorted['exit_time'], df_sorted['drawdown'], linewidth=1, color='red')
        axes[1].axhline(y=max_drawdown, color='darkred', linestyle='--', 
                       label=f'Max DD: {max_drawdown:.1f}%')
        axes[1].set_title('Drawdown Analysis', fontweight='bold', fontsize=12)
        axes[1].set_xlabel('Date')
        axes[1].set_ylabel('Drawdown (%)')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        axes[1].tick_params(axis='x', rotation=45)
    else:
        axes[1].axis('off')
    
    # Profit Distribution with dynamic bins
    if show_profit_dist:
        n_bins = min(50, len(df) // 5)
        axes[2].hist(df['profit'], bins=n_bins, alpha=0.7, color='skyblue', 
                    edgecolor='black', density=True)
        axes[2].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Break-even')
        axes[2].axvline(x=avg_trade, color='green', linestyle='--', linewidth=2, 
                       label=f'Avg: ${avg_trade:.2f}')
        axes[2].set_title('Profit/Loss Distribution', fontweight='bold', fontsize=12)
        axes[2].set_xlabel('Profit/Loss ($)')
        axes[2].set_ylabel('Density')
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)
    else:
        axes[2].axis('off')
    
    # Rolling Performance
    if window_size > 0 and show_duration_dist:
        rolling_profit = df_sorted['profit'].rolling(window=window_size).sum()
        rolling_win_rate = (df_sorted['profit'] > 0).rolling(window=window_size).mean() * 100
        
        ax_twin = axes[3].twinx()
        line1 = axes[3].plot(df_sorted['exit_time'], rolling_profit, 'b-', linewidth=2, 
                           label=f'Rolling {window_size}-trade P&L')
        line2 = ax_twin.plot(df_sorted['exit_time'], rolling_win_rate, 'r-', linewidth=2, 
                           label=f'Rolling {window_size}-trade Win Rate')
        
        axes[3].set_xlabel('Date')
        axes[3].set_ylabel('Rolling P&L ($)', color='b')
        ax_twin.set_ylabel('Win Rate (%)', color='r')
        axes[3].set_title(f'Rolling {window_size}-Trade Performance', fontweight='bold', fontsize=12)
        axes[3].grid(True, alpha=0.3)
        axes[3].tick_params(axis='x', rotation=45)
        
        # Combine legends
        lines = line1 + line2
        labels = [l.get_label() for l in lines]
        axes[3].legend(lines, labels, loc='upper left')
    else:
        axes[3].axis('off')
    
    plt.tight_layout()
    plt.show()

print("🎮 Interactive Visualizations - Use controls below:")
interact(create_interactive_charts,
         show_equity=True,
         show_drawdown=True, 
         show_profit_dist=True,
         show_duration_dist=True,
         window_size=IntSlider(value=30, min=5, max=100, step=5, description='Rolling Window:'))

# %% [markdown]
# ## 2. ADVANCED TRADE ANALYSIS

# %%
def analyze_trades_by_filter(analysis_type='hour', metric='profit', top_n=10):
    """Interactive trade analysis by different categories"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    if analysis_type == 'hour':
        group_col = 'entry_hour'
        x_label = 'Hour of Day'
    elif analysis_type == 'direction':
        group_col = 'direction'
        x_label = 'Trade Direction'
    elif analysis_type == 'exit_reason':
        group_col = 'exit_reason'
        reason_counts = df['exit_reason'].value_counts()
        top_reasons = reason_counts.head(top_n).index
        df_filtered = df[df['exit_reason'].isin(top_reasons)]
        x_label = 'Exit Reason'
    elif analysis_type == 'weekday':
        group_col = 'entry_weekday'
        x_label = 'Day of Week'
    elif analysis_type == 'month':
        group_col = 'entry_month'
        x_label = 'Month'
    
    # Group data based on analysis type
    if analysis_type == 'exit_reason':
        grouped = df_filtered.groupby(group_col)
    else:
        grouped = df.groupby(group_col)
    
    # Calculate metrics based on selection
    if metric == 'profit':
        summary = grouped['profit'].mean()
        title_suffix = 'Average Profit'
        color = 'blue'
        y_label = 'Average Profit ($)'
    elif metric == 'win_rate':
        summary = grouped['profit'].apply(lambda x: (x > 0).mean() * 100)
        title_suffix = 'Win Rate'
        color = 'green'
        y_label = 'Win Rate (%)'
    elif metric == 'count':
        summary = grouped.size()
        title_suffix = 'Trade Count'
        color = 'orange'
        y_label = 'Number of Trades'
    elif metric == 'expectancy':
        summary = grouped.apply(lambda x: (len(x[x['profit'] > 0])/len(x) * x[x['profit'] > 0]['profit'].mean() + 
                                         len(x[x['profit'] < 0])/len(x) * x[x['profit'] < 0]['profit'].mean()) 
                                         if len(x) > 0 else 0)
        title_suffix = 'Expectancy'
        color = 'purple'
        y_label = 'Expectancy ($)'
    
    # Ensure we have all hours for hourly analysis
    if analysis_type == 'hour':
        full_index = pd.Index(range(24), name='entry_hour')
        summary = summary.reindex(full_index).fillna(0)
    elif analysis_type == 'weekday':
        weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        summary.index = [weekday_names[i] for i in summary.index]
    
    # Create bar plot
    x_positions = range(len(summary))
    bars = ax1.bar(x_positions, summary.values, alpha=0.7, color=color)
    
    if analysis_type == 'hour':
        ax1.set_xticks(x_positions)
        ax1.set_xticklabels(range(24))
    else:
        ax1.set_xticks(x_positions)
        ax1.set_xticklabels(summary.index, rotation=45)
    
    ax1.set_title(f'{title_suffix} by {x_label}', fontweight='bold', fontsize=12)
    ax1.set_xlabel(x_label)
    ax1.set_ylabel(y_label)
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, summary.values):
        if metric == 'profit':
            label = f'${value:.1f}' if abs(value) < 100 else f'${value:.0f}'
        elif metric == 'win_rate':
            label = f'{value:.1f}%'
        elif metric == 'expectancy':
            label = f'${value:.1f}'
        else:
            label = f'{value:.0f}'
        
        height = bar.get_height()
        y_pos = height + 0.02 * abs(height) if height >= 0 else height - 0.02 * abs(height)
        va = 'bottom' if height >= 0 else 'top'
            
        ax1.text(bar.get_x() + bar.get_width()/2, y_pos, label,
                ha='center', va=va, fontweight='bold', fontsize=9)
    
    # Second plot
    if metric == 'profit' and len(summary) > 0:
        best_category = summary.idxmax()
        if analysis_type == 'hour':
            category_data = df[df['entry_hour'] == best_category]['profit']
            category_name = f'Hour {best_category}'
        elif analysis_type == 'direction':
            category_data = df[df['direction'] == best_category]['profit']
            category_name = best_category.capitalize()
        elif analysis_type == 'exit_reason':
            category_data = df[df['exit_reason'] == best_category]['profit']
            category_name = best_category
        elif analysis_type == 'weekday':
            category_data = df[df['entry_weekday'] == best_category]['profit']
            category_name = f'Weekday {best_category}'
        elif analysis_type == 'month':
            category_data = df[df['entry_month'] == best_category]['profit']
            category_name = f'Month {best_category}'
        
        if len(category_data) > 0:
            ax2.hist(category_data, bins=min(20, len(category_data)//2), 
                    alpha=0.7, color='lightgreen', edgecolor='black')
            ax2.axvline(x=category_data.mean(), color='red', linestyle='--', 
                       label=f'Avg: ${category_data.mean():.2f}')
            ax2.axvline(x=0, color='black', linestyle='-', alpha=0.3)
            ax2.set_title(f'Profit Distribution: {category_name}\n({len(category_data)} trades)', 
                         fontweight='bold', fontsize=12)
            ax2.set_xlabel('Profit/Loss ($)')
            ax2.set_ylabel('Frequency')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
        else:
            ax2.text(0.5, 0.5, 'No data for best category', 
                    ha='center', va='center', transform=ax2.transAxes, fontsize=12)
            ax2.set_title('Distribution Analysis', fontweight='bold', fontsize=12)
    else:
        ax2.text(0.5, 0.5, f'Analysis Complete', 
                ha='center', va='center', transform=ax2.transAxes, fontsize=12)
        ax2.set_title('Analysis Summary', fontweight='bold', fontsize=12)
        ax2.set_xticks([])
        ax2.set_yticks([])
    
    plt.tight_layout()
    plt.show()

print("🎮 Advanced Trade Analysis - Use controls below:")
interact(analyze_trades_by_filter,
         analysis_type=Dropdown(options=['hour', 'direction', 'exit_reason', 'weekday', 'month'], 
                               value='hour', description='Analyze by:'),
         metric=Dropdown(options=['profit', 'win_rate', 'count', 'expectancy'],
                        value='profit', description='Metric:'),
         top_n=IntSlider(value=8, min=3, max=15, description='Top Reasons:'))

# %% [markdown]
# ## 3. STATISTICAL ANALYSIS DASHBOARD

# %%
def statistical_analysis(show_distribution=True, show_autocorr=True, show_rolling_stats=True, 
                        correlation_lag=10, rolling_window=50):
    """Advanced statistical analysis of trading performance"""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Returns Distribution with Statistical Tests
    if show_distribution:
        n_bins = min(40, len(returns))
        axes[0,0].hist(returns * 100, bins=n_bins, alpha=0.7, color='lightblue', 
                      edgecolor='black', density=True)
        
        # Fit normal distribution
        x = np.linspace(returns.min() * 100, returns.max() * 100, 100)
        normal_pdf = norm.pdf(x, returns.mean() * 100, returns.std() * 100)
        axes[0,0].plot(x, normal_pdf, 'r-', linewidth=2, label='Normal Distribution')
        
        axes[0,0].axvline(x=returns.mean() * 100, color='green', linestyle='--', 
                         label=f'Mean: {returns.mean()*100:.3f}%')
        axes[0,0].set_title('Return Distribution with Statistical Tests', fontweight='bold')
        axes[0,0].set_xlabel('Return (%)')
        axes[0,0].set_ylabel('Density')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)
        
        # Add statistical annotations
        stats_text = f'Skewness: {returns_skewness:.3f}\nKurtosis: {returns_kurtosis:.3f}\nJB p-value: {jb_pvalue:.4f}'
        axes[0,0].text(0.02, 0.98, stats_text, transform=axes[0,0].transAxes, 
                      verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Autocorrelation Analysis
    if show_autocorr:
        from statsmodels.tsa.stattools import acf
        lags = range(1, correlation_lag + 1)
        autocorrs = [acf(returns, nlags=lag)[-1] for lag in lags]
        
        axes[0,1].bar(lags, autocorrs, alpha=0.7, color='lightgreen')
        axes[0,1].axhline(y=0, color='red', linestyle='-', alpha=0.5)
        axes[0,1].axhline(y=2/np.sqrt(len(returns)), color='red', linestyle='--', alpha=0.5, label='95% CI')
        axes[0,1].axhline(y=-2/np.sqrt(len(returns)), color='red', linestyle='--', alpha=0.5)
        axes[0,1].set_title('Returns Autocorrelation', fontweight='bold')
        axes[0,1].set_xlabel('Lag')
        axes[0,1].set_ylabel('Autocorrelation')
        axes[0,1].legend()
        axes[0,1].grid(True, alpha=0.3)
    
    # Rolling Statistics
    if show_rolling_stats:
        rolling_mean = df_sorted['profit'].rolling(window=rolling_window).mean()
        rolling_std = df_sorted['profit'].rolling(window=rolling_window).std()
        rolling_sharpe = rolling_mean / rolling_std * np.sqrt(252) if rolling_std.mean() > 0 else 0
        
        ax_twin = axes[1,0].twinx()
        line1 = axes[1,0].plot(df_sorted['exit_time'], rolling_mean, 'b-', linewidth=2, 
                              label=f'Rolling {rolling_window}-trade Mean')
        line2 = ax_twin.plot(df_sorted['exit_time'], rolling_sharpe, 'r-', linewidth=2, 
                            label=f'Rolling {rolling_window}-trade Sharpe')
        
        axes[1,0].set_xlabel('Date')
        axes[1,0].set_ylabel('Rolling Mean ($)', color='b')
        ax_twin.set_ylabel('Rolling Sharpe Ratio', color='r')
        axes[1,0].set_title(f'Rolling Statistics (Window: {rolling_window})', fontweight='bold')
        axes[1,0].grid(True, alpha=0.3)
        axes[1,0].tick_params(axis='x', rotation=45)
        
        lines = line1 + line2
        labels = [l.get_label() for l in lines]
        axes[1,0].legend(lines, labels, loc='upper left')
    
    # Win/Loss Streak Analysis
    df_sorted['result'] = np.where(df_sorted['profit'] > 0, 1, -1)
    df_sorted['streak'] = df_sorted['result'].groupby((df_sorted['result'] != df_sorted['result'].shift()).cumsum()).cumcount() + 1
    df_sorted['streak_id'] = (df_sorted['result'] != df_sorted['result'].shift()).cumsum()
    
    winning_streaks = df_sorted[df_sorted['result'] == 1].groupby('streak_id')['streak'].max()
    losing_streaks = df_sorted[df_sorted['result'] == -1].groupby('streak_id')['streak'].max()
    
    if len(winning_streaks) > 0 and len(losing_streaks) > 0:
        axes[1,1].hist([winning_streaks, losing_streaks], bins=range(1, max(max(winning_streaks), max(losing_streaks)) + 2), 
                      alpha=0.7, color=['lightgreen', 'lightcoral'], label=['Winning Streaks', 'Losing Streaks'], 
                      stacked=True)
        axes[1,1].set_title('Win/Loss Streak Distribution', fontweight='bold')
        axes[1,1].set_xlabel('Streak Length')
        axes[1,1].set_ylabel('Frequency')
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistical summary
    t_stat, p_value = ttest_1samp(returns, 0)
    
    print(f"""
📊 STATISTICAL SUMMARY:
• Skewness: {returns_skewness:.3f} {'(Right-skewed)' if returns_skewness > 0 else '(Left-skewed)'}
• Kurtosis: {returns_kurtosis:.3f} {'(Leptokurtic)' if returns_kurtosis > 0 else '(Platykurtic)'}
• Jarque-Bera p-value: {jb_pvalue:.4f} {'(Normal)' if jb_pvalue > 0.05 else '(Non-normal)'}
• T-test p-value: {p_value:.4f} {'(Significant)' if p_value < 0.05 else '(Not significant)'}
• Largest Win: ${largest_win:.2f}
• Largest Loss: ${largest_loss:.2f}
• Ulcer Index: {ulcer_index:.2f}
""")

print("🎮 Statistical Analysis - Use controls below:")
interact(statistical_analysis,
         show_distribution=True,
         show_autocorr=True,
         show_rolling_stats=True,
         correlation_lag=IntSlider(value=10, min=5, max=50, description='Correlation Lags:'),
         rolling_window=IntSlider(value=50, min=10, max=200, description='Rolling Window:'))

# %% [markdown]
# ## 4. RISK ANALYSIS & MONTE CARLO SIMULATION

# %%
def calculate_risk_metrics(confidence_level=0.95, max_slippage_pct=10, num_simulations=500):
    """Combined risk analysis and Monte Carlo simulation"""
    
    # Risk Analysis
    var_historical = np.percentile(returns, (1 - confidence_level) * 100)
    var_parametric = norm.ppf(1 - confidence_level, returns.mean(), returns.std())
    es_historical = returns[returns <= var_historical].mean()
    
    # Monte Carlo Simulation
    def monte_carlo_single_simulation(df, max_slippage_pct=10, num_simulations=1000):
        initial_capital = df['open_equity'].iloc[0]
        final_results = []
        drawdowns_list = []
        all_equity_curves = []
        
        np.random.seed(42)
        
        for sim in range(num_simulations):
            shuffled_trades = df.sample(frac=1, replace=False, random_state=sim)
            adjusted_profits = []
            
            for _, trade in shuffled_trades.iterrows():
                base_profit = trade['profit']
                if max_slippage_pct > 0:
                    actual_slippage = np.random.uniform(-max_slippage_pct, max_slippage_pct)
                    slippage_effect = base_profit * (actual_slippage / 100)
                else:
                    slippage_effect = 0
                
                adjusted_profits.append(base_profit + slippage_effect)
            
            equity_curve = initial_capital + np.cumsum(adjusted_profits)
            final_equity = equity_curve[-1]
            
            max_equity = np.maximum.accumulate(equity_curve)
            drawdown = (equity_curve - max_equity) / max_equity * 100
            max_dd = np.min(drawdown) if len(drawdown) > 0 else 0
            
            final_results.append(final_equity)
            drawdowns_list.append(max_dd)
            all_equity_curves.append(equity_curve)
        
        return np.array(final_results), np.array(drawdowns_list), all_equity_curves
    
    # Run simulation
    final_equities, max_drawdowns, equity_curves = monte_carlo_single_simulation(
        df, max_slippage_pct=max_slippage_pct, num_simulations=num_simulations
    )
    
    avg_final = final_equities.mean()
    prob_profit = (final_equities > initial_balance).mean() * 100
    var_95 = np.percentile(final_equities, 5)
    avg_max_drawdown = max_drawdowns.mean()
    
    print(f"""
📊 RISK METRICS (Confidence: {confidence_level:.0%}):
• Historical VaR: {var_historical:.3%}
• Parametric VaR: {var_parametric:.3%}
• Expected Shortfall: {es_historical:.3%}
• 95% VaR: {var_95:.3%}
• 99% VaR: {var_99:.3%}

📊 MONTE CARLO RESULTS (Max Slippage: ±{max_slippage_pct}%, Simulations: {num_simulations}):
• Average Final Equity: ${avg_final:,.2f}
• Probability of Profit: {prob_profit:.1f}%
• 5% VaR: ${np.percentile(final_equities, 5):,.2f}
• Average Max Drawdown: {avg_max_drawdown:.2f}%
• Best Case: ${np.percentile(final_equities, 95):,.2f}
• Worst Case: ${np.percentile(final_equities, 5):,.2f}
""")
    
    # Visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # VaR Comparison
    methods = ['Historical', 'Parametric']
    var_values = [var_historical, var_parametric]
    
    bars = ax1.bar(methods, [abs(v) for v in var_values], alpha=0.7, color=['lightcoral', 'lightblue'])
    ax1.set_title(f'Value at Risk ({confidence_level:.0%} Confidence)', fontweight='bold')
    ax1.set_ylabel('VaR (Absolute Value)')
    ax1.grid(True, alpha=0.3)
    
    for bar, value in zip(bars, var_values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{value:.3%}',
                ha='center', va='bottom', fontweight='bold')
    
    # Risk-Return Distribution
    n_bins = min(40, len(returns))
    ax2.hist(returns * 100, bins=n_bins, alpha=0.7, color='lightgreen', edgecolor='black', density=True)
    ax2.axvline(x=var_historical * 100, color='red', linestyle='--', linewidth=2, 
               label=f'Historical VaR: {var_historical:.3%}')
    ax2.axvline(x=returns.mean() * 100, color='blue', linestyle='--', linewidth=2,
               label=f'Mean Return: {returns.mean():.3%}')
    ax2.set_title('Return Distribution with Risk Metrics', fontweight='bold')
    ax2.set_xlabel('Return (%)')
    ax2.set_ylabel('Density')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Monte Carlo Paths
    for i, curve in enumerate(equity_curves[:30]):
        ax3.plot(curve, alpha=0.1, color='blue')
    ax3.axhline(y=initial_balance, color='black', linestyle='--', linewidth=2, 
               label='Initial Capital')
    ax3.set_title(f'Monte Carlo Simulation Paths\n(Max Slippage: ±{max_slippage_pct}%)', fontweight='bold')
    ax3.set_xlabel('Trade Sequence')
    ax3.set_ylabel('Equity ($)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Final Equity Distribution
    ax4.hist(final_equities, bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
    ax4.axvline(x=initial_balance, color='red', linestyle='--', linewidth=2,
               label='Initial Capital')
    ax4.axvline(x=avg_final, color='blue', linestyle='--', linewidth=2,
               label=f'Average: ${avg_final:,.0f}')
    ax4.axvline(x=var_95, color='orange', linestyle='--', linewidth=2,
               label=f'5% VaR: ${var_95:,.0f}')
    ax4.set_title('Final Equity Distribution', fontweight='bold')
    ax4.set_xlabel('Final Equity ($)')
    ax4.set_ylabel('Frequency')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("🎮 Risk & Monte Carlo Analysis - Use controls below:")
interact(calculate_risk_metrics,
         confidence_level=FloatSlider(value=0.95, min=0.90, max=0.99, step=0.01, 
                                    description='Confidence Level:'),
         max_slippage_pct=FloatSlider(value=1.0, min=0.0, max=5.0, step=0.1, 
                                description='Max Slippage %:'),
         num_simulations=IntSlider(value=500, min=100, max=2000, step=100,
                                 description='Simulations:'))

# %% [markdown]
# ## 5. COMPREHENSIVE STRATEGY ASSESSMENT

# %%
def generate_final_report(risk_tolerance='medium'):
    """Generate final report with risk-adjusted rating"""
    
    if risk_tolerance == 'low':
        pf_threshold = 1.8
        sharpe_threshold = 1.2
        dd_threshold = -15
        win_rate_threshold = 60
    elif risk_tolerance == 'medium':
        pf_threshold = 1.5
        sharpe_threshold = 1.0
        dd_threshold = -20
        win_rate_threshold = 55
    else:
        pf_threshold = 1.2
        sharpe_threshold = 0.7
        dd_threshold = -25
        win_rate_threshold = 50
    
    score = 0
    if profit_factor > pf_threshold: score += 1
    if sharpe_ratio > sharpe_threshold: score += 1
    if max_drawdown > dd_threshold: score += 1
    if winning_percentage > win_rate_threshold: score += 1
    if expectancy > 0: score += 1
    if kelly_fraction > 0: score += 1
    
    if score >= 5:
        rating = "🟢 EXCELLENT"
        recommendation = "Strategy shows outstanding performance across all metrics"
    elif score >= 4:
        rating = "🟡 VERY GOOD" 
        recommendation = "Strong strategy with minor areas for optimization"
    elif score >= 3:
        rating = "🟠 GOOD"
        recommendation = "Solid strategy with some areas for improvement"
    else:
        rating = "🔴 NEEDS IMPROVEMENT"
        recommendation = "Consider significant optimization of strategy parameters"
    
    print(f"""
🎯 COMPREHENSIVE STRATEGY ASSESSMENT (Risk Tolerance: {risk_tolerance.upper()})
{'='*80}
Rating: {rating}
{recommendation}

📊 PERFORMANCE SUMMARY:
• Total Return: {total_return:.2f}% (${net_profit:+,.0f}) {'✅' if total_return > 0 else '❌'}
• Profit Factor: {profit_factor:.2f} {'✅' if profit_factor > pf_threshold else '❌'}
• Win Rate: {winning_percentage:.1f}% {'✅' if winning_percentage > win_rate_threshold else '❌'} 
• Sharpe Ratio: {sharpe_ratio:.2f} {'✅' if sharpe_ratio > sharpe_threshold else '❌'}
• Max Drawdown: {max_drawdown:.2f}% {'✅' if max_drawdown > dd_threshold else '❌'}
• Expectancy: ${expectancy:.2f} {'✅' if expectancy > 0 else '❌'}
• Kelly Fraction: {kelly_fraction:.2%} {'✅' if kelly_fraction > 0 else '❌'}

📈 ADVANCED METRICS:
• Sortino Ratio: {sortino_ratio:.2f}
• Calmar Ratio: {calmar_ratio:.2f}
• Win/Loss Ratio: {win_loss_ratio:.2f}
• Ulcer Index: {ulcer_index:.2f}
• 95% VaR: {var_95:.3%}
• Expected Shortfall: {cvar_95:.3%}

💡 STRATEGY RECOMMENDATIONS:
{'• Consider position sizing optimization using Kelly criterion' if kelly_fraction > 0 else '• Avoid trading - negative expectancy detected'}
{'• Focus on improving win rate consistency' if winning_percentage < 50 else '• Maintain current win rate discipline'}
{'• Implement drawdown control measures' if max_drawdown < -25 else '• Drawdown management is effective'}
{'• Diversify strategy to reduce risk' if sharpe_ratio < 0.5 else '• Risk-adjusted returns are strong'}
{'• Excellent strategy - consider scaling carefully' if score >= 5 else ''}

🎯 SCORE: {score}/6
""")

print("🎮 Comprehensive Strategy Assessment - Use controls below:")
interact(generate_final_report,
         risk_tolerance=Dropdown(options=['low', 'medium', 'high'], 
                               value='medium', description='Risk Tolerance:'))

# Export comprehensive results
summary_data = {
    'Metric': ['Total Return %', 'Net Profit', 'Profit Factor', 'Win Rate %', 'Max Drawdown %', 
               'Sharpe Ratio', 'Sortino Ratio', 'Expectancy', 'Kelly Fraction', 'Win/Loss Ratio',
               '95% VaR', 'Ulcer Index', 'Total Trades', 'Winning Trades', 'Losing Trades'],
    'Value': [f"{total_return:.2f}%", f"${net_profit:,.2f}", f"{profit_factor:.2f}", 
              f"{winning_percentage:.1f}%", f"{max_drawdown:.2f}%", f"{sharpe_ratio:.2f}",
              f"{sortino_ratio:.2f}", f"${expectancy:.2f}", f"{kelly_fraction:.2%}",
              f"{win_loss_ratio:.2f}", f"{var_95:.3%}", f"{ulcer_index:.2f}",
              f"{len(df)}", f"{winning_trades}", f"{losing_trades}"]
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('comprehensive_trading_performance_summary.csv', index=False)
print("✅ Comprehensive results exported to 'comprehensive_trading_performance_summary.csv'")

print(f"""
🎉 COMPREHENSIVE ANALYSIS COMPLETE!
• Analyzed {len(df)} trades with {winning_trades} wins and {losing_trades} losses
• Total return: {total_return:.2f}% (${net_profit:+,.0f})
• Strategy expectancy: ${expectancy:.2f} per trade
• Optimal position size: {kelly_fraction:.2%} of capital
• Use the interactive controls in each section to explore the strategy in depth
""")

📊 Loading and preprocessing data...
✅ Loaded 514 transactions

🎯 KEY PERFORMANCE METRICS:
• Total Return: 4.43%
• Net Profit: $443.30
• Profit Factor: 3.75
• Win Rate: 75.9%
• Max Drawdown: -0.06%
• Sharpe Ratio: 11.22

📊 ADVANCED METRICS:
• Expectancy: $0.86
• Win/Loss Ratio: 1.19
• Kelly Fraction: 42.80%
• Sortino Ratio: 3593339722346519.00
• 95% VaR: -0.013%

🎮 Interactive Visualizations - Use controls below:


interactive(children=(Checkbox(value=True, description='show_equity'), Checkbox(value=True, description='show_…

🎮 Advanced Trade Analysis - Use controls below:


interactive(children=(Dropdown(description='Analyze by:', options=('hour', 'direction', 'exit_reason', 'weekda…

🎮 Statistical Analysis - Use controls below:


...

🎮 Risk & Monte Carlo Analysis - Use controls below:


interactive(children=(FloatSlider(value=0.95, description='Confidence Level:', max=0.99, min=0.9, step=0.01), …

🎮 Comprehensive Strategy Assessment - Use controls below:


interactive(children=(Dropdown(description='Risk Tolerance:', index=1, options=('low', 'medium', 'high'), valu…

✅ Comprehensive results exported to 'comprehensive_trading_performance_summary.csv'

🎉 COMPREHENSIVE ANALYSIS COMPLETE!
• Analyzed 514 trades with 390 wins and 124 losses
• Total return: 4.43% ($+443)
• Strategy expectancy: $0.86 per trade
• Optimal position size: 42.80% of capital
• Use the interactive controls in each section to explore the strategy in depth

